# Minimization and Fitting

## Scipy

In this lecture, we will start to explore some practical problems that can be solved by applying algorithms to some data. Python provides several libraries to perform complex scientific and technical operations, and [SciPy](https://scipy.github.io/devdocs/index.html) is one of the most complete available libraries.

SciPy contains modules for optimization, linear algebra, integration, interpolation, special functions, FFT, signal and image processing, ODE solvers and other tasks common in science and engineering.

Available sub-packages include:

- `constants`: physical constants and conversion factors
- `optimize`: optimization algorithms including linear programming (today)
- `interpolate`: interpolation tools (today)
- `stats`: statistical functions (today and in the next lecture)
- `integrate`: numerical integration routines (in the next lecture)
- `fft`: Discrete Fourier Transform algorithms (in the next lecture)
- `fftpack`: Legacy interface for Discrete Fourier Transforms (in the next lecture)
- `linalg`: linear algebra routines (in the next lectures)
- `sparse`: sparse matrices and related algorithms (in the next lectures)
- and many others


For what concerns us, the `scipy.optimize` module provides several algorithms for function minimization (scalar or multi-dimensional), curve fitting and root finding.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

%matplotlib inline

##  Roots of functions

There are several root finding algorithms for a generic function $f(x): {\rm I\!R} \to {\rm I\!R}$, e.g. [Newton-Raphson](https://en.wikipedia.org/wiki/Newton%27s_method) or Brent (a safer version of the secant method, which combines a bracketing strategy with a parabolic approximation)

In [ ]:
def f(x):
    return x**3-3*x+1

x = np.linspace(-3, 3, 100)
plt.axhline(0)
plt.plot(x, f(x),'r-')

In [ ]:
from scipy.optimize import newton
# The Newton method just needs the initial point
x1, x2, x3 = newton(f, -3), newton(f, 0), newton(f, 3)
print(x1, x2, x3)

In [ ]:
%timeit newton(f, 0)

Let's try now the [Brentq](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html) module from ScipPy:

In [ ]:
from scipy.optimize import brentq
# Specifying the function and the range to look for zeroes is mandatory
x1, x2, x3 = brentq(f, -3, 0), brentq(f, 0, 1), brentq(f, 1,3)
print(x1, x2, x3)

In [ ]:
%timeit brentq(f, 0, 1)

As you can verify, two independent methods yield the same result, but the performance can be very different, as every algorithm has its pros and cons.

In case of $f(x): {\rm I\!R}^2 \to {\rm I\!R}$, the `optimize.root` is available, also implementing several methods (MINPACK's hybrid being the default).

When performing complex root finding and minimizations, always **check if the algorithm has converged!**

In [ ]:
from scipy.optimize import root, fsolve

def f(x):
    return [x[1] - 3*x[0]*(x[0]+1)*(x[0]-1), .25*x[0]**2 + x[1]**2 - 1]

solutions = root(f, (0.5, 0.5))
# optimize.root returns an object which contains the result of the root finding
print(solutions)

print("\n\nSolution:\nx0 =", solutions['x'], "\nf(x0) =", f(solutions['x']))

## Minimization of a function

In general, minimization of $f(x): {\rm I\!R}^n \to {\rm I\!R}$ is a whole field of study per se. You should have seen it treated in details in other courses.

In the example below, we see an example of how SciPy can perform the numerical minimization of a function.
The `minimize_scalar` function will find the minimum, and a search interval can be specified. By default, it uses the Brent algorithm.

In [ ]:
def f(x):
    return x**4 + 3*(x-2)**3 - 15*(x)**2 + 1

x = np.linspace(-8, 5, 100)
plt.plot(x, f(x))

In [ ]:
solution = optimize.minimize_scalar(f, method='Brent')
print(solution)

In [ ]:
solution = optimize.minimize_scalar(f, method='bounded', bounds=[0, 6])
print(solution) # different algorithms return different objects

#### Local and global minima

If the function is not trivial (and especially if it is in higher dimensions than 1), it may contain several minima. In these cases, care has to be taken to specify a good starting point, otherwise the algorithm will converge in a local (false) minimum.

In [ ]:
def f(x, offset):
    return -np.sinc(x - offset)

x = np.linspace(-20, 20, 100)
plt.plot(x, f(x, 5))

In [ ]:
# note how additional function arguments are passed in
solution = optimize.minimize_scalar(f, args=(5,))
print(solution)

Optimization results are packed accordingly to this [format](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html#scipy.optimize.OptimizeResult).

In [ ]:
plt.plot(x, f(x, 5))
plt.axvline(solution['x'], color='b')

When the function oscillates a lot, often the global minimum is not found. To cope with that, several ranges needs to be explored. This can be done either manually...

In [ ]:
lower = np.random.uniform(-20, 20, 100)
upper = lower + 1
solutions = [optimize.minimize_scalar(f, args=(5,), bracket=(l, u)) for (l, u) in zip(lower, upper)]
# split the x range in intervals, and run minimize_scalar in each one, then pick the interval with the smallest minimum
idx = np.argmin([solution.fun for solution in solutions])
solution = solutions[idx]

plt.plot(x, f(x, 5))
plt.axvline(solution['x'], color='r')

...or using dedicated methods which are specifically designed to overcome these issues:

In [ ]:
from scipy.optimize import basinhopping

x0 = -10
solution = basinhopping(f, x0, stepsize=1, minimizer_kwargs={'args': (5,)}) # see how the parameters are passed to the function
print(solution)

In [ ]:
plt.plot(x, f(x, 5))
plt.axvline(solution['x'], color='r')

It's very likely that an algorithm that solves your problem has already been implemented: in this case, it's always more efficient to use the dedicated library.

## Interpolation

Sometimes, it is useful for interpolating some data and thus evaluating intermediate points where no measure exists, without performing any fit. The `scipy.interpolate` module is based on the FITPACK Fortran subroutines.

The kind of interpolation can be specified: linear is the default;  *zero*, *slinear*, *quadratic* and *cubic* refer to a [spline](https://en.wikipedia.org/wiki/Spline_(mathematics)) interpolation of zeroth, first, second or third order; *previous* and *next* simply return the previous or next value of the point.

In [ ]:
# the actual measurements
measured_time = np.linspace(0, 1, 10)
noise = (np.random.random(10)*2 - 1) * 1.e-1
measures = np.sin(2 * np.pi * measured_time) + noise

# import scipy.interpolate
from scipy.interpolate import interp1d
# 1D linear interpolation
linear_interp = interp1d(measured_time, measures)
# cubic interpolation
cubic_interp = interp1d(measured_time, measures, kind='cubic')

# denser interval to plot the interpolating function
interpolation_time = np.linspace(0, 1, 50)

plt.figure(figsize=(6, 4))
# plot analytic function
plt.plot(measured_time, measures, 'o', ms=6, label='measures')
# plot linear interpolation
linear_results = linear_interp(interpolation_time)
plt.plot(interpolation_time, linear_results, label='linear interp')
# plot cubic interpolation
cubic_results = cubic_interp(interpolation_time)
plt.plot(interpolation_time, cubic_results, label='cubic interp')
plt.legend()

# checking the punctual results
x = 0.2
print("sin(", x, "):", np.sin(2 * np.pi * x), "\n",
       "linear prediction:",linear_interp(x), "\n",
       "cubic prediction:", cubic_interp(x))

## Linear Regression

We have some data, $x$ and $y$, and we suppose there is a linear relation between them.
Thus, we want to fit a line to the data. Our model will be:

$$\hat{y} = \hat{\alpha} + \hat{\beta}x$$

where the symbol $\hat{}$ indicates our best estimate of that parameter.

For the sake of clarity, we define the following symbols:

* $\alpha$: the intercept
* $\hat{\alpha}$: our estimated best intercept
* $\beta$: the slope
* $\hat{\beta}$: our estimated best slope
* $\hat{y}$: our estimated y's, or in other words $f(x, \hat{\alpha}, \hat{\beta})$
* $x,y$: the data
* $\bar{x}$, $\bar{y}$: the average of $x,y$
* Residual: the difference between $\hat{y}$ and $y$ at a particular point ($\hat{y}_i - y_i$)
* Sum of squared residues (SSR): $\sum_i (\hat{y}_i - y_i)^2$
* Total sum of squares (TSS): $\sum_i (\bar{y} - y_i)^2$, which is a kind of variance

One way to view this problem is as a minimization. Specifically, we want to have a function that returns a value of how good the estimate of our parameters is. As a matter of fact, our ultimate goal is to make $\hat{y}$ as close as possible to $y$. One of the most immediate choices is to take $\sum_i(\hat{y}_i - y_i)^2$, and minimize the resulting quantity. 
So we need to write down a function which takes in $\hat{\alpha}$ and $\hat{\beta}$ and returns how good the fit is:

$$f(\hat{\alpha}, \hat{\beta}) = \sum_i \left(y_i - \hat{\alpha} - \hat{\beta} x_i\right)^2$$

We can minimize this equation using any of our minimization techniques or we can do it analytically. In case of a linear regression, this can be easily done analytically.

Using calculus you can show that the minimum to $f(\alpha, \beta)$ is:

$$\hat{\beta} = \frac{\sum_i(x_i - \bar{x})(y_i - \bar{y})}{\sum_i(x_i - \bar{x})^2}$$

With a little bit of algebra, you can show this is

$$\hat{\beta} = \frac{\sigma_{xy}}{\sigma_x^2}$$

where $\sigma_{xy}$ is the sample covariance of $x$ and $y$ and $\sigma_x^2$ is the sample variance of $x$.

To find the intercept, you can just take the average of the residuals (not their squares!) given the model so far:

$$\hat{\alpha} = \frac{1}{N}\sum_i (y_i - \hat{\beta}x_i)$$

The uncertainties of $\hat{\alpha}$, $\hat{\beta}$ can be derived as well, but they strongly depend on the assumptions that you do on the uncertainties of the single points.


Under the assumption that the data points are normally distributed with mean $\hat{\beta}$ and variance $\sigma^2 / \sum (x_i - \hat{x})^2$, where $\sigma^2$ is the variance of the error terms, the uncertainties on the parameters $\alpha$ and $\beta$ can be derived as:

$$\sigma_\hat{\beta}^2 = \frac{\frac{SSR}{n-2}}{\sum (x_i - \bar{x})^2}$$

$$\sigma_\hat{\alpha}^2 = \sigma_\hat{\beta}^2 \frac{\sum x_i^2}{n}$$

Note that if you have a prior on the uncertainty on the single points, these uncertainties will not be valid and should be updated with the single point uncertainty terms!

Let's see the regression in action:

In [ ]:
# generate some data with a linear relation + add some random noise
x = np.linspace(0, 10, 20)
y = 1 + x * 2.5 + np.random.normal(scale=2, size=20)

# get the covariance. Numpy conveniently provides the np.cov() method
cov = np.cov(x, y, ddof=2)
b_hat = cov[0, 1] / cov[0, 0]
a_hat = np.mean(y - b_hat * x)

n = len(x)
ssr = np.sum((y - (a_hat + b_hat * x))**2)
sigma_b = np.sqrt( (ssr/(n-2)) / np.sum((x - np.mean(x))**2) )
sigma_a = np.sqrt( sigma_b**2 * np.sum(x**2) / n )

print("fitted alpha = %.2f +- %.2f" % (a_hat, sigma_a))
print("fitted beta  = %.2f +- %.2f" % (b_hat, sigma_b))
print("covariance:\n", cov)

# plot resulting line
plt.plot(x, y, 'o')
plt.plot(x, a_hat + b_hat * x)
plt.show()

Notice that we didn't get exactly the correct answer: the points were generated with a slope of 2.5 and an intercept of 1, whereas our fit was a little bit off. But taking into account the uncertainties on the parameters, we can see that they are compatible with the initial values.

## Goodness of Fit

There are a several ways to asses whether the function chosen fits well the data. There are different ways to perform a "goodness of fit". Two common methods are:

#### The correlation coefficient

One viable and quick way is to just compute the SSR, which stands for *sum of the squared residuals*. However, the SSR has the downside that it depends on the unit of $y$. Therefore, it's a good idea to divide the SSR by the TSS (*total sum of squares*), which has the same unit, so the result is a pure number:

$$R^2 = 1 - \frac{\textrm{SSR}}{\textrm{TSS}} = 1 - \frac{\sum_i \left(\hat{y}_i - y\right)^2}{\sum_i \left(\bar{y} - y\right)^2}$$

This equation has the property that it is unitless, and it is $1$ when the fit is perfect, and $0$ when the fit does not describe the data at all. In the case of linear regression, $R$ is the same as the correlation coefficient.

In the case of the previous fit:

In [ ]:
ssr = np.sum((y - a_hat - b_hat * x)**2)
tss = np.sum((np.mean(y) - y)**2)

rsq = 1 - ssr / tss

print("R2 =", rsq, "\nR =", sqrt(rsq))
print("Covariance matrix:\n", np.corrcoef(x,y)) # check with the correlation matrix that R is the correlation coefficient

#### The p-value

A proper goodness-of-fit test involves more than just some simple numerical estimator. The $\chi^2$ goodness of fit test is used to determine whether the sample data is consistent with a hypothesized distribution (in this case, a linear dependence, but it works for all data models). The $\chi^2$ goodness of fit test is appropriate when the data is randomly distributed, and the number of observations is not small, e.g. $\geq 5$.

In order to perform the test, we need to:

- determine the number of degrees of freedom of the fit, which is the number of data points - 1
- choose a figure of merit of how "good" the fit is, in this case the $\chi^2$, defined as:
$$\chi^2 = \sum_i \frac{(y_i - \hat{y}_i)^2}{\sigma_i^2}$$
- determine the **p-value**, which is the probability of observing a sample statistic as extreme as the test statistic. The $\chi^2$ as test statistics is a good choice, because it has a known distribution given the number of degrees of freedom and under the assumption that the data is randomly distributed. Thus, the p-value is the integral of the $\chi^2$ from the value returned from the fit to $+\infty$. When no scientific libraries were available, these values were tabulated, but nowadays `scipy.stats` provides a lot of these functions (including their cumulative distribution functions, which allow to quickly calculate the integral)

In [ ]:
p = np.arange(0, 30, 0.1)
plt.plot(p, scipy.stats.chi2.pdf(p, df=4)) # very small p-value: model likely doesn't fit data
plt.plot(p, scipy.stats.chi2.pdf(p, df=8))
plt.plot(p, scipy.stats.chi2.pdf(p, df=12)) # very large p-value:
plt.axvline(12.7, color='r')

In our example, let's now assume (in order to use the $\chi^2$) that the data represents counts, whose uncertainty is known from the Poisson pdf and are simply $\sigma_{y_i} = \sqrt{y_i}$:

In [ ]:
sigma_y = np.sqrt(np.abs(y)) # assume random counts, e.g. sigma_i = sqrt(y_i). This is not always the case!

plt.errorbar(x, y, yerr=sigma_y, fmt='.k') # use plt.errorbar to plot points with the corresponding uncertainties

In [ ]:
ndof = len(x) - 1
print("n degrees of freedom =", ndof)
# calculate the chi^2
chi2 = np.sum(((y - (a_hat + b_hat * x))**2) / sigma_y)
print("chi2 =", chi2)
# calculate the p-value from the chi^2, the n.d.o.f., and the comulative chi^2 distribution
pvalue = 1. - scipy.stats.chi2.cdf(chi2, ndof)
print("p-value =", pvalue) # if the p-value is < 0.05, the fit is considered unsatisfactory

plt.plot(p, scipy.stats.chi2.cdf(p, df=ndof))
plt.fill_between(x, y, where=(x > chi2), color='lightblue')
plt.axvline(chi2, color='r')

## Curve Fitting

Unfortunately, not all datasets have a linear dependence like the ones we have seen before. Often, there is a more complicated function that needs to be fitted to the data. In these cases, you may:

1) apply an appropriate transform to the data to get back to the linear regression case (if applicable), or

2) perform a full fit with the desired function to the data

In this example, we will see how to do the latter. Be aware that there are more than one test statistics that can be chosen (it could be simply $\sum (\hat{y}_i - y_i)^2$, or the $\chi^2$, Likelihood, ...).

The best estimate of the function parameters are then determined through the numerical minimization of the test statistics.

Let's create some data, sampled from a sine distribution and affected by random noise:

In [ ]:
x_data = np.linspace(-5, 5, num=50)
y_data = 2.9 * np.sin(1.5 * x_data) + np.random.normal(size=50)

plt.figure(figsize=(6, 4))
plt.scatter(x_data, y_data)

This case is very straightfoward, because we *know* that the data is sampled from a sine function.

To find its parameters, in this case amplitude and (angular) frequency, we can call the `optimize.curve_fit()`, which accepts as arguments the fitting function, the data, and [much more](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html). 

In [ ]:
def f(x, A, nu):
    return A * np.sin(nu * x) # sine function with amplitude A and angular frequency

# fit by means of curve_fit
# p0 is a list of the initial guessed values for the parameters
popt, pcov = optimize.curve_fit(f, x_data, y_data, p0=[2, 2], full_output=False)
print("parameters:\n", params)
print("covariance matrix:\n", params_covariance)
print("A = %.2f +- %.2f" % (popt[0], np.sqrt(pcov[0,0])))
print("f = %.2f +- %.2f" % (popt[1], np.sqrt(pcov[1,1])))

Note: the way curve_fit determines the uncertainty is to actually renormalize the errors so that the reduced $\chi^2$ (that is, the $\chi^2$/number of degrees of freedom) value is one, so the magnitude of the errors doesn't matter, only the relative errors. In some cases, when you have a prior on your uncertainites, you can specify `absolute_sigma=True` and pass an appropriate array containing the uncertainties in order to preserve the original errors.

In [ ]:
plt.scatter(x_data, y_data, label='data')
plt.plot(x_data, f(x_data, params[0], params[1]), label='fitted function')
plt.legend(loc='best')

## The Fisher F-test

Fitting is one of the crucial points of data analysis or in general data science. If you *know* the underlying model, then the overall problem is simple; if you don't, then it is a very delicate issue.

In some lucky cases, your data is distributed according to a known relation (a linear dependence on time, or the exponential decrease of a certain count) you can infer from your knowledge of the problem you are dealing with.

In all other cases, you don't know which function should be used for fitting, because you don't have the full knowledge of the underlying problem. In order to find a function, a common practice is to proceed by trial-and-error, although this poses several problems such that the choice of the function is arbitrary, and there is the risk to over/under-fit the data.

In [ ]:
from IPython.display import Image
Image(filename = 'xkcd_curve_fitting.png') 

The method called **Fisher F-test** [[1](http://EconPapers.repec.org/RePEc:bla:istatr:v:80:y:2012:i:3:p:491-491)] [[2](https://doi.org/10.2307/2340521)] can help to determine the correct number of parameters in an appropriate family of functions (e.g. polynomials, exponentials, ...). From the statistical point of view, this method is not trivial at all because you need a prior on the choice of the function.

Please note that if you don't even have the knowledge of which family of functions to be used to fit the data, then the problem is more complex and the F-test is not applicable.

Instead, if you have a prior on the family of the function but not the exact number of parameters, the Fisher F-test provides an useful recipe.

The F-test is based on some fit quantities, such as the RSS (squared sum of the residuals of the fit) and the number of degrees of freedom. The test uses these quantities to iteratively compare one reference function and the function with one more parameter to determine whether or not the additional parameter is needed. If the simpler fit function is correct, the relative increase in the sum of squares is expected to be smaller than the relative increase in DOF:

$$\frac{SSR_1 - SSR_2}{SSR_2} \lesssim \frac{n_1 - n_2}{n_2}$$

the F-test is based on the test statistics that is defined as follows:

$$F = \left( \frac{SSR_1 - SSR_2}{n_2 - n_1} \right) / \left( \frac{SSR_2}{N - n_2} \right)$$

The F value is used to determine a CL after integrating the Fisher distribution from $F$ to $+\infty$, with $N$ the number of total degrees of freedom (bins):

$$CL = 1 - \int_0^F \mathcal{F} d\mathcal{f}$$

which gives the CL under the null hypothesis of the simpler function being sufficient. If $CL < 10\%$, the simpler function is sufficient, otherwise, more parameters are needed. In the latter case, the Fisher test is repeated by increasing the number of parameters of both functions, until a function with a sufficient number of parameters is found.

In [ ]:
from scipy.stats import f


# define function for F-test
def Ftest(ssr_1, ssr_2, ndof_1, ndof_2, nbins, verbose=False):
    F = ((ssr_1 - ssr_2)/(ndof_2 - ndof_1)) / (ssr_2/(nbins - ndof_2))
    CL = 1. - f.cdf(F, ndof_2 - ndof_1, nbins - ndof_2)
    if verbose: print("CL: %.3f" % CL, ", additional parameter necessary:", "YES" if CL < 0.10 else "NO")
    return CL


# generate data according to a certain distribution + random noise
N = 100
x = np.linspace(0, 5., N)
rnd = np.random.normal(size=(N))
y = x**2 + rnd

# plot
fig, ax = plt.subplots(figsize=(10, 6)) # create the figure and the axes
ax.scatter(x=x, y=y, marker='o', c='darkblue', label="data")

# define fitting functions
def f1(x, a0, a1):
    return a0 + a1 * x

def f2(x, a0, a1, a2):
    return a0 + a1 * x + a2 * x**2

def f3(x, a0, a1, a2, a3):
    return a0 + a1 * x + a2 * x**2 + a3 * x**3

(fa0, fa1), cov = optimize.curve_fit(f1, x, y)
y1 = f1(x, fa0, fa1)
ax.plot(x, y1, color='r', label="linear")
ssr_1 = np.sum((y1 - y)**2)

(fa0, fa1, fa2), cov = optimize.curve_fit(f2, x, y)
y2 = f2(x, fa0, fa1, fa2)
ax.plot(x, y2, color='g', label="quadratic")
ssr_2 = np.sum((y2 - y)**2)

(fa0, fa1, fa2, fa3), cov = optimize.curve_fit(f3, x, y)
y3 = f3(x, fa0, fa1, fa2, fa3)
ax.plot(x, y3, color='y', label="cubic")
ssr_3 = np.sum((y3 - y)**2)

ax.legend(loc=(0.65, 0.8))

print("SSR:", ssr_1, ssr_2, ssr_3)

# run F-test
# test f2 vs f1
print("Test Linear vs Quadratic:")
cl_1_vs_2 = Ftest(ssr_1, ssr_2, 1, 2, N, verbose=True)

print("Test Quadratic vs Cubic:")
cl_2_vs_3 = Ftest(ssr_2, ssr_3, 2, 3, N, verbose=True)